# <div align="center">Variational Autoencoders (VAE)</div>
---------------------------------------------------------------------

you can Find me on Github:
> ###### [ GitHub](https://github.com/lev1khachatryan)


***Useful links:***

Original paper http://arxiv.org/abs/1312.6114
  
  
Helpful videos explaining the topic
https://www.youtube.com/watch?v=P78QYjWh5sM

http://videolectures.net/deeplearning2015_courville_autoencoder_extension/?q=aaron%20courville

In a nutshell, a VAE is an autoencoder whose encodings distribution is regularised during the training in order to ensure that its latent space has good properties allowing us to generate some new data. Moreover, the term ***variational*** comes from the close relation between the regularisation and the variational inference method in statistics.

But many questions are raises.

1. How to generate new data from VAEs? 


2. What is the link between VAEs and variational inference? 

...

In order to describe VAEs as well as possible, we will try to answer all this questions (and many others!) ranging from basic intuitions to more advanced mathematical details.

## Limitations of autoencoders for content generation

A natural question that comes in mind is ***what is the link between autoencoders and content generation?***. Indeed, once the autoencoder has been trained, we have both an encoder and a decoder but still no real way to produce any new content. At first sight, we could be tempted to think that, if the latent space is regular enough, we could take a point randomly from that latent space and decode it to get a new content. The decoder would then act more or less like the generator of a Generative Adversarial Network.

However, the regularity of the latent space for autoencoders is a difficult point that depends on the distribution of the data in the initial space, the dimension of the latent space and the architecture of the encoder. So, it is pretty difficult (if not impossible) to ensure, a priori, that the encoder will organize the latent space in a smart way compatible with the generative process.

## Definition of variational autoencoders

So, in order to be able to use the decoder of our autoencoder for generative purpose, we have to be sure that the latent space is regular enough. One possible solution to obtain such regularity is to introduce explicit regularisation during the training process. Thus, a variational autoencoder can be defined as being an autoencoder whose training is regularised to avoid overfitting and ensure that the latent space has good properties that enable generative process.

Just as a standard autoencoder, a variational autoencoder is an architecture composed of both an encoder and a decoder and that is trained to minimise the reconstruction error between the encoded-decoded data and the initial data. However, in order to introduce some regularisation of the latent space, we proceed to a slight modification of the encoding-decoding process: 

instead of encoding an input as a single point, we ***encode it as a distribution over the latent space***. The model is then trained as follows:

* first, the input is encoded as distribution over the latent space


* second, a point from the latent space is sampled from that distribution


* third, the sampled point is decoded and the reconstruction error can be computed


* finally, the reconstruction error is backpropagated through the network

<img src='assets/20191127/2.png'>

In practice, the encoded distributions are chosen to be normal so that the encoder can be trained to return the mean and the covariance matrix that describe these Gaussians. The reason why an input is encoded as a distribution with some variance instead of a single point is that it makes possible to express very naturally the ***latent space regularisation***: the distributions returned by the encoder are enforced to be close to a standard normal distribution. We will see in the next subsection that we ensure this way both a local and global regularisation of the latent space (local because of the variance control and global because of the mean control).

Thus, the loss function that is minimised when training a VAE is composed of a ***reconstruction term*** (on the final layer), that tends to make the encoding-decoding scheme as performant as possible, and a ***regularisation term*** (on the latent layer), that tends to regularise the organisation of the latent space by making the distributions returned by the encoder close to a standard normal distribution. That regularisation term is expressed as the Kulback-Leibler divergence between the returned distribution and a standard Gaussian and will be further justified in the next section. We can notice that the Kullback-Leibler divergence between two Gaussian distributions has a closed form that can be directly expressed in terms of the means and the covariance matrices of the the two distributions.

<img src='assets/20191127/3.png'>

## Intuitions about the regularisation

The regularity that is expected from the latent space in order to make generative process possible can be expressed through two main properties: ***continuity*** (two close points in the latent space should not give two completely different contents once decoded) and ***completeness*** (for a chosen distribution, a point sampled from the latent space should give “meaningful” content once decoded).

The only fact that VAEs encode inputs as distributions instead of simple points is not sufficient to ensure continuity and completeness. Without a well defined regularisation term, the model can learn, in order to minimise its reconstruction error, to “ignore” the fact that distributions are returned and behave almost like classic autoencoders (leading to overfitting). To do so, the encoder can either return distributions with tiny variances (that would tend to be punctual distributions) or return distributions with very different means (that would then be really far apart from each other in the latent space). In both cases, distributions are used the wrong way (cancelling the expected benefit) and continuity and/or completeness are not satisfied.

So, in order to avoid these effects we have to ***regularise both the covariance matrix and the mean of the distributions returned by the encoder***. In practice, this regularisation is done by enforcing distributions to be close to a standard normal distribution (centred and reduced). This way, we require the covariance matrices to be close to the identity, preventing punctual distributions, and the mean to be close to 0, preventing encoded distributions to be too far apart from each others.

<img src="assets/20191127/4.png">

With this regularisation term, we prevent the model to encode data far apart in the latent space and encourage as much as possible returned distributions to “overlap”, satisfying this way the expected continuity and completeness conditions. Naturally, as for any regularisation term, this comes at the price of a higher reconstruction error on the training data. The tradeoff between the reconstruction error and the KL divergence can however be adjusted and we will see in the next section how the expression of the balance naturally emerge from our formal derivation.

To conclude this subsection, we can observe that continuity and completeness obtained with regularisation tend to create a ***gradient*** over the information encoded in the latent space. For example, a point of the latent space that would be halfway between the means of two encoded distributions coming from different training data should be decoded in something that is somewhere between the data that gave the first distribution and the data that gave the second distribution as it may be sampled by the autoencoder in both cases.

<img src="assets/20191127/5.png">

# Mathematical details of VAEs

In the previous section we gave the following intuitive overview: VAEs are autoencoders that encode inputs as distributions instead of points and whose latent space “organisation” is regularised by constraining distributions returned by the encoder to be close to a standard Gaussian. In this section we will give a more mathematical view of VAEs that will allow us to justify the regularisation term more rigorously. To do so, we will set a clear probabilistic framework and will use, in particular, variational inference technique.



### Probabilistic framework and assumptions

Let’s begin by defining a probabilistic graphical model to describe our data. We denote by $x$ the variable that represents our data and assume that $x$ is generated from a latent variable $z$ (the encoded representation) that is not directly observed. Thus, for each data point, the following two steps generative process is assumed:

* first, a latent representation $z$ is sampled from the prior distribution $p(z)$

* second, the data $x$ is sampled from the conditional likelihood distribution $p(x|z)$

With such a probabilistic model in mind, we can redefine our notions of encoder and decoder. Indeed, contrarily to a simple autoencoder that consider ***deterministic*** encoder and decoder, we are going to consider now ***probabilistic*** versions of these two objects. The ***probabilistic decoder*** is naturally defined by ***p(x|z)***, that describes the distribution of the decoded variable given the encoded one, whereas the ***probabilistic encoder*** is defined by ***p(z|x)***, that describes the distribution of the encoded variable given the decoded one.

At this point, we can already notice that the regularisation of the latent space that we lacked in simple autoencoders naturally appears here in the definition of the data generation process: encoded representations $z$ in the latent space are indeed assumed to follow the prior distribution $p(z)$. Otherwise, we can also remind the well-known Bayes theorem that makes the link between the prior $p(z)$, the likelihood $p(x|z)$, and the posterior $p(z|x)$

<img src='assets/20191127/6.png'>

Let’s now make the assumption that $p(z)$ is a ***standard Gaussian distribution*** and that $p(x|z)$ is a Gaussian distribution whose mean is defined by a deterministic function f of the variable of z and whose covariance matrix has the form of a positive constant c that multiplies the identity matrix I. The function f is assumed to belong to a family of functions denoted F that is left unspecified for the moment and that will be chosen later. Thus, we have

<img src='assets/20191127/7.png'>

Let’s consider, for now, that f is well defined and fixed. In theory, as we know $p(z)$ and $p(x|z)$, we can use the Bayes theorem to compute $p(z|x)$: this is a classical Bayesian inference problem. However, as we discussed in our previous article, this kind of computation is often intractable (because of the integral at the denominator) and require the use of approximation techniques such as variational inference.

***Note.*** Here we can mention that $p(z)$ and $p(x|z)$ are both Gaussian distributions, implying that $p(z|x)$ should also follow a Gaussian distribution. In theory, we could then “only” try to express the mean and the covariance matrix of $p(z|x)$ with respect to the means and the covariance matrices of $p(z)$ and $p(x|z)$. However, in practice these values depend on the function $f$ that can be complex and that is not defined for now (even if we have assumed the contrary). Moreover, the use of an approximation technique like variational inference makes the approach pretty general and more robust to some changes in the hypothesis of the model.


## Variational inference formulation

In statistics, variational inference (VI) is a technique to approximate ***complex distributions***. The idea is to set a parametrised family of distribution (for example the family of Gaussians, whose parameters are the mean and the covariance) and to look for the best approximation of our target distribution among this family. The best element in the family is one that minimise a given approximation error measurement (most of the time the Kullback-Leibler divergence between approximation and target) and is found by gradient descent over the parameters that describe the family. For more details, you can read [this](https://towardsdatascience.com/bayesian-inference-problem-mcmc-and-variational-inference-25a8aa9bce29) post.

Here we are going to approximate $p(z|x)$ by a Gaussian distribution $q_x(z)$ whose mean and covariance are defined by two functions, ***g*** and ***h***, of the parameter ***x***. These two functions are supposed to belong, respectively, to the families of functions ***G*** and ***H*** that will be specified later but that are supposed to be parametrised. Thus we can denote

<img src='assets/20191127/8.png'>

So, we have defined this way a family of candidates for variational inference and need now to find the best approximation among this family by optimising the functions g and h (in fact, their parameters) to minimise the ***Kullback-Leibler divergence*** between the approximation and the target p(z|x). In other words, we are looking for the optimal g* and h* such that

<img src='assets/20191127/9.png'>

In the second last equation, we can observe the tradeoff there exists — when approximating the posterior ***p(z|x)*** — between maximising the likelihood of the “observations” (maximisation of the expected log-likelihood, for the first term) and staying close to the prior distribution (minimisation of the KL divergence between q_x(z) and p(z), for the second term). This tradeoff is natural for Bayesian inference problem and express the balance that needs to be found between the confidence we have in the data and the confidence we have in the prior.

